# Charity Register Data Scrape
## Louis Othen
The purpose of this script is to scrape of all charities found on the Charity register owned by the governments Charity Commission. the register can be found here: 
https://www.gov.uk/find-charity-information

The reason for the scrape is to collect information on a charity, what it does, what contact information it has, and format into a table. This table will be joined with another previous created before sending a mass email out to them to offer my volunteering services in data.

# Import the relevant libraries

In [3]:
import pandas as pd

from selenium                       import webdriver
from selenium.webdriver.common.by   import By
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup as bs
from tqdm import tqdm 
import requests as req
import time as tim
from lxml import etree
#from ydata_profiling import ProfileReport  #Stopped working since python 3.12
import re
from plotly import express as px
from datetime import datetime as dti

## Inital Scrape

### Set up with selenium to get past cookies

In [37]:
# Open up test browser window
#-----------------------------------------------------
ded = webdriver.Chrome()

# Navigate to charity register screen 
#-----------------------------------------------------
ded.get('https://register-of-charities.charitycommission.gov.uk/charity-search/-/results/page/1/delta/20/sorted-by/charity-income/desc')

# Accept cookies
#-----------------------------------------------------
cookies_btn = ded.find_element(By.ID,'_com_placecube_cookieconsent_web_portlet_CookieConsentPortlet_acceptAllCookies')
cookies_btn.click()

In [38]:
# Get list of page numbers
#-----------------------------------------------------
end_page_len = ded.find_elements(By.ID, "_uk_gov_ccew_portlet_CharitySearchPortlet_pageIterator")

# Get value of last page number 
#-----------------------------------------------------
pe = end_page_len[0].text.split(sep='\n')
end_page_num = int(pe[-2])

### Using requests to parse the contents of each page

In [ ]:
header = {
            "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
            "X-Requested-With": "XMLHttpRequest"
        }

# Empty list ready to store looped data 
#-----------------------------------------------------
collected_data  = []

# For loop over webpages, to download high level charity data
#-----------------------------------------------------
for i in tqdm(range(1,end_page_num)):
    url_use = 'https://register-of-charities.charitycommission.gov.uk/charity-search/-/results/page/' + str(i) + '/delta/20/sorted-by/charity-income/desc'#
    print(url_use)
    r= req.get(url_use, headers = header)
    collected_data.append(pd.read_html(r.text)[0])

# Save collected list as a pandas DataFrame
#-----------------------------------------------------
result = pd.concat(collected_data)

# Remove unneeded columns
#-----------------------------------------------------
result.drop(columns=['level_0', 'index'], inplace=True)
    

In [ ]:
# See results of scrape before saving down into CSV file.
#-----------------------------------------------------
#result = pd.concat(collected_data)
display(result)
result.to_csv('D:\Coding\Charity_Scrape\charityregister.csv')

## Now to get the child data from each individual link  

### Set up dataframe with columns to mine from webpages

In [ ]:
# Read in previously farmed dataframe of summary pages
#-----------------------------------------------------
result = pd.read_csv('D:\Coding\Charity_Scrape\charityregister.csv')

# Remove records with Nulls - not useful at this stage - may be worth looking at later
#-----------------------------------------------------
result = result.dropna()

result.drop(columns = ['Unnamed: 0'], inplace = True) # weird index as per previous download -  remove

# Set up columns ready for further mining
#-----------------------------------------------------
result['Charity number'] = result['Charity number'].astype(str).str.split('.').str[0]
result['overview'] = ''
result['what_it_does'] = ''
result['who_it_benefits'] = ''
result['how_its_done'] = ''
result['where_they_operate'] = ''
result['address'] = ''
result['phone'] = ''
result['email'] = ''
result['website'] = ''

# get a sample to play with
testo = result


In [ ]:
# function to automate 1) get link and content , and prep for xpath extraction
#-----------------------------------------------------
def get_page(url):
    header =    {
                    "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
                    "X-Requested-With": "XMLHttpRequest"
                }
    try:
        
        # page = req.session()
        # page.cookies.update({'__hs_opt_out': 'no'})

        url = url
        charity_page = req.get(url=url_use, headers=header)

        soup = bs(charity_page.text,'lxml')
        dom = etree.HTML(str(soup))
    
        return dom
    
    except requests.exceptions.RequestException as e:
        print(f"Error fetching page: {e}")
        return None

In [ ]:
# Loop through each charity details and store it 

for index, row in tqdm(testo.iterrows(), total = testo.shape[0]):
     url_use = 'https://register-of-charities.charitycommission.gov.uk/charity-search/-/charity-details/' + row['Charity number']

     xpage = get_page(url_use)
     try:
          testo.loc[index, 'overview']            = xpage.xpath('//*[@id="_uk_gov_ccew_onereg_charitydetails_web_portlet_CharityDetailsPortlet_mainContent"]/div[1]/div/p')[0].text
     except IndexError:
          testo.loc[index, 'overview']            = "N/A"

     url_use = url_use + '/what-who-how-where'
     wpage = get_page(url_use)

     try:
          testo.loc[index, 'what_it_does']        = wpage.xpath('//*[@id="_uk_gov_ccew_onereg_charitydetails_web_portlet_CharityDetailsPortlet_mainContent"]/div[2]/div[2]/ul/li')[0].text
          testo.loc[index, 'who_it_benefits']     = wpage.xpath('//*[@id="_uk_gov_ccew_onereg_charitydetails_web_portlet_CharityDetailsPortlet_mainContent"]/div[3]/div[2]/ul/li')[0].text
          testo.at[index, 'how_its_done']         = wpage.xpath('//*[@id="_uk_gov_ccew_onereg_charitydetails_web_portlet_CharityDetailsPortlet_mainContent"]/div[4]/div[2]/ul/li/text()')
          testo.at[index, 'where_they_operate']   = wpage.xpath('//*[@id="_uk_gov_ccew_onereg_charitydetails_web_portlet_CharityDetailsPortlet_mainContent"]/div[5]/div[2]/ul/li/text()')
     except IndexError:
          testo.loc[index, 'what_it_does']        = "N/A"
          testo.loc[index, 'who_it_benefits']     = "N/A"
          testo.loc[index, 'how_its_done']        = "N/A"
          testo.loc[index, 'where_they_operate']  = "N/A"
     url_use = url_use.replace('/what-who-how-where', '')

     url_use = url_use + '/contact-information'
     cpage = get_page(url_use)


     try:
          testo.at[index, 'address']              = cpage.xpath('//*[@id="_uk_gov_ccew_onereg_charitydetails_web_portlet_CharityDetailsPortlet_mainContent"]/dl/div[1]/dd/text()')
          testo.loc[index, 'phone']               = cpage.xpath('//*[@id="_uk_gov_ccew_onereg_charitydetails_web_portlet_CharityDetailsPortlet_mainContent"]/dl/div[2]/dd/text()')
                                                                 
          testo.loc[index, 'email']               = cpage.xpath('//*[@id="charity-contact-email-link"]/text()')
          testo.loc[index, 'website']             = cpage.xpath('//*[@id="charity-contact-website-link"]/text()')
     except Exception:
          testo.loc[index, 'address']             = "N/A"
          testo.loc[index, 'phone']               = "N/A"
          testo.loc[index, 'email']               = "N/A"
          testo.loc[index, 'website']             = "N/A"
     
     tim.sleep(0.05)

     

     

Save this Data!

In [ ]:
testo.to_csv('D:\Coding\Charity_Scrape\charityregister_detail_raw.csv')

## Data Cleaning
now that all the data we need has been farmed / mined, next phase is to clean things up before performing Exploratory data analysis upon it

 bring the data back in 

In [69]:
raw_data = pd.read_csv("D:\Coding\Charity_Scrape\charityregister_detail_raw.csv")

clean = raw_data
clean.dropna(
                            subset = [
                                        'overview'
                                        ,'what_it_does'
                                        ,'who_it_benefits'
                                        ,'how_its_done'
                                        ,'where_they_operate'
                                        ,'address'
                                        ,'phone'
                                        ,'email'
                                        ,'website'
                                     ]
                           ,inplace= True
                        )
clean.reset_index(inplace = True)
clean.drop(columns=['Unnamed: 0', 'Reporting', 'Status','how_its_done','where_they_operate','Charity number','index'], inplace=True)

C:\Users\lothe\AppData\Local\Temp\ipykernel_18724\700711209.py:1: DtypeWarning: Columns (6,7,8,9,10,11,12,13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  raw_data = pd.read_csv("D:\Coding\Automation\charityregister_detail_raw.csv")


In [78]:
clean['overview'] = clean['overview'].str.replace('\n', '').str.replace('\t', '').str.replace("'",'')

# Removed or now as messy from mining, but could use later
#--------------------------------------------------------------------------------------
#clean['how_its_done'] = clean['how_its_done'].str.replace('[', '').str.replace("'",'')
#clean['how_its_done'] = clean['how_its_done'].str.replace(']', '').str.replace("'",'')    
#clean['where_they_operate'] = clean['where_they_operate'].str.replace('[', '').str.replace(']', '').str.replace("'",'')

clean['website'] = clean['website'].str.replace('[','').str.replace(']', '').str.replace("'",'')
clean['email'] = clean['email'].str.replace('[','').str.replace(']', '').str.replace("'",'')
clean['phone'] = clean['phone'].apply(lambda x: re.findall(r'\d+', str(x))).apply(lambda x: ''.join(map(str,x)))
clean['address'] = clean['address'].apply(lambda x: re.findall(r'\w+\d*\s+',str(x))).apply(lambda x: [item.replace('t ', '')  if item == 't ' else item for item in x ]).apply(lambda x: ' '.join(x)).apply(lambda x: re.sub(r'\s+', ' ',x).strip())

clean['Income'] = clean['Income'].str.replace('£','').str.replace(',','')

clean = clean.applymap(lambda x: x.lower())

In [79]:
clean.head()

,Charity name,Income,overview,what_it_does,who_it_benefits,address,phone,email,website
0,the charities aid foundation,1044275000,charities aid foundation (caf) is a registered...,general charitable purposes,other charities or voluntary bodies,charities aid foundation 25 kings hill avenue ...,03000123088,companysecretary@cafonline.org,www.cafonline.org
1,nuffield health,995600000,"to advance, promote and maintain health and he...",children/young people,provides services,nuffield health epsom gateway 2 ashley avenue ...,07824482211,iben.thomson@nuffieldhealth.com,www.nuffieldhealth.com
2,the arts council of england,941845043,arts council england works to get great art to...,arts/culture/heritage/science,other charities or voluntary bodies,arts council england the hive 49 lever street ...,01619344317,enquiries@artscouncil.org.uk,www.artscouncil.org.uk
3,the british council,896655974,the british council creates friendly knowledge...,the general public/mankind,makes grants to individuals,1 redman place stratford london united kingdom,01619577755,ceo.office@britishcouncil.org,www.britishcouncil.org
4,the national trust for places of historic inte...,643329000,to look after places of historic interest or n...,the general public/mankind,provides buildings/facilities/open space,national trust heelis kemble drive swindon,01793817400,thesecretary@nationaltrust.org.uk,www.nationaltrust.org.uk


In [42]:
# Currently not working since updating python to 3.12
#ProfileReport(clean)

# Lets save it for more stuff later ! 


In [80]:
clean.to_csv("D:\Coding\Charity_Scrape\charityregister_detail_clean.csv")

# Read it back in for analysis of what we want to go for 

In [8]:
data = pd.read_csv("D:\Coding\Charity_Scrape\charityregister_detail_clean.csv")

In [9]:
data.head()
data.drop(columns=['Unnamed: 0'], inplace=True)

bring previous charity data and join them together


In [11]:
prev_data = pd.read_csv("D:\Coding\Charity_Scrape\charity_email_list.csv")
final_data = pd.concat([data, prev_data])
final_data.columns

Index(['Charity name', 'Income', 'overview', 'what_it_does', 'who_it_benefits',
       'address', 'phone', 'email', 'website'],
      dtype='object')

In [13]:
final_data

,Charity name,Income,overview,what_it_does,email,website
0,the charities aid foundation,1044275000,charities aid foundation (caf) is a registered...,general charitable purposes,companysecretary@cafonline.org,www.cafonline.org
1,nuffield health,995600000,"to advance, promote and maintain health and he...",children/young people,iben.thomson@nuffieldhealth.com,www.nuffieldhealth.com
2,the arts council of england,941845043,arts council england works to get great art to...,arts/culture/heritage/science,enquiries@artscouncil.org.uk,www.artscouncil.org.uk
3,the british council,896655974,the british council creates friendly knowledge...,the general public/mankind,ceo.office@britishcouncil.org,www.britishcouncil.org
4,the national trust for places of historic inte...,643329000,to look after places of historic interest or n...,the general public/mankind,thesecretary@nationaltrust.org.uk,www.nationaltrust.org.uk
...,...,...,...,...,...,...
13,Macmillan,0,cancer support,cancer support,contact@macmillan.org.uk,https://www.macmillan.org.uk/
14,Revitalise,0,repsirte for carer and disabled,repsirte for carer and disabled,info@revitalise.org.uk,https://revitalise.org.uk
15,The Earl of Southampton Trust,0,local housing for the elderly,local housing for the elderly,info@eost.org.uk,https://eost.org.uk
16,Dogs Trust,0,Dog Rehoming,Dog Rehoming,info@dogstrust.org.uk,https://www.dogstrust.org.uk


In [16]:
final_data.to_csv("D:\Coding\Charity_Scrape\charity_email_merged.csv")

# Analyse what is there and remove unwanted

In [17]:
df = pd.read_csv("D:\Coding\Charity_Scrape\charity_email_merged.csv")
df.drop(columns=['Unnamed: 0'], inplace=True)
df.columns
df.shape

(12144, 6)

Get rid of more charities within the category chosen, such as religious or partisan related. Essentially, avoid anything that could be contrived as 'controversial'.


In [25]:
words_to_remove = ['islam', 'islamic','diocese','synagogue','ARTHRITIS','HORSE','CHURCH','GOD','Religion','Catholic','MOTORISTS','Christian'
                   ,'SPIRITUAL','COAL','Gospel','THEATRE','Missionary','VEGAN','CINEMA','evange','football','DANCE','choir'
                   ,'estate','trains','Holocaust','ROMAN','SPORTS','GALLERY','opera','CLUB','POLISH','MANOR','LEISURE','worship'
                   ,'chapel','cats','BUDDHIST','League','Sisters','museum','sheep','breeders','BOTANICAL','NAUTICAL', 'BUILDINGS'
                   ,'SIKH','chinese','ART','AGRICULTURAL','HALL','DIOCESAN','PRODUCTIONS','COLLECTION','PALACE'
                   ,'ORCHESTRA','WHALE', 'DOLPHIN','Christian','BIBLE','CHRIST','theological','Jesus','festival']

cols_to_search = ['Charity name']

row_rm = c_data[cols_to_search].apply(lambda x: x.str.contains('|'.join(words_to_remove), case=False, na=False)).any(axis=1)

c_data = c_data[~row_rm]

c_data.shape

(9231, 6)

In [54]:
# Save final filtered data ready to choose charities to approach
#-----------------------------------------------------
c_data.to_csv("D:\Coding\Automation\charity_email_final.csv")

# Now to look at charities with a particular condition to focus on.


In [4]:
c_data = pd.read_csv("D:/Coding/git_projects/Charity_Scrape/charity_email_final.csv")
display(c_data)

,Charity name,Income,overview,what_it_does,email,website
0,the grange centre for people with disabilities,5002085,to maximise the independence of adults with ph...,people with disabilities,info@grangecentre.org.uk,www.grangecentre.org.uk
1,brambletye school trust limited,5008530,education/training of young people,children/young people,schooloffice@brambletye.com,www.brambletye.co.uk
2,treetops hospice trust,5016230,"treetops hospice provides daycare, complementa...",people with disabilities,info@treetopshospice.org.uk,www.treetopshospice.org.uk
3,the hammond school limited,5024030,the hammond school is one of 8 vocational seco...,children/young people,info@thehammondschool.co.uk,www.thehammondschool.co.uk
4,groundwork yorkshire limited,5055289,to bring about lasting environmental improveme...,children/young people,wakefield@groundwork.org.uk,https://www.groundwork.org.uk/hubs/north-east-...
...,...,...,...,...,...,...
828,garfield weston foundation,84004000,the garfield weston foundation is a family-fou...,general charitable purposes,admin@garfieldweston.org,www.garfieldweston.org
829,the royal national institute of blind people,85709000,rnib is the uks leading charity offering infor...,children/young people,helpline@rnib.org.uk,www.rnib.org.uk
830,education development trust,89070000,the charity seeks to advance education for the...,children/young people,corpgov@educationdevelopmenttrust.com,http://www.educationdevelopmenttrust.com/
831,wateraid,91913000,"wateraid is an international not-for-profit, d...",the prevention or relief of poverty,supportercare@wateraid.org,www.wateraid.org


In [5]:
c_types = set(c_data['what_it_does'])
c_types

{'amateur sport',
 'arts/culture/heritage/science',
 'children/young people',
 'disability',
 'education/training',
 'elderly/old people',
 'environment/conservation/heritage',
 'general charitable purposes',
 'human rights/religious or racial harmony/equality or diversity',
 'other charities or voluntary bodies',
 'other defined groups',
 'people of a particular ethnic or racial origin',
 'people with disabilities',
 'religious activities',
 'the advancement of health or saving of lives',
 'the prevention or relief of poverty'}

In [7]:
to_send = c_data.head(n = 100)

to_send

,Charity name,Income,overview,what_it_does,email,website
0,the grange centre for people with disabilities,5002085,to maximise the independence of adults with ph...,people with disabilities,info@grangecentre.org.uk,www.grangecentre.org.uk
1,brambletye school trust limited,5008530,education/training of young people,children/young people,schooloffice@brambletye.com,www.brambletye.co.uk
2,treetops hospice trust,5016230,"treetops hospice provides daycare, complementa...",people with disabilities,info@treetopshospice.org.uk,www.treetopshospice.org.uk
3,the hammond school limited,5024030,the hammond school is one of 8 vocational seco...,children/young people,info@thehammondschool.co.uk,www.thehammondschool.co.uk
4,groundwork yorkshire limited,5055289,to bring about lasting environmental improveme...,children/young people,wakefield@groundwork.org.uk,https://www.groundwork.org.uk/hubs/north-east-...
...,...,...,...,...,...,...
95,kingsway clc trust,5769301,to advance the christian faith through the pro...,children/young people,sales@kingswayclctrust.com,http://www.kingswayclctrust.com
96,rikkyo school in england trust,5785251,the education in accordance with the christian...,children/young people,eikoku@rikkyo.uk,www.rikkyo.co.uk
97,british home and hospital for incurables,5788669,british home provides specialist nursing care ...,people with disabilities,enquiries@britishhome.org.uk,www.britishhome.org.uk
98,milton keynes community foundation limited,5798052,we are a local independant grant making charit...,general charitable purposes,info@mkcommunityfoundation.co.uk,www.mkcommunityfoundation.co.uk


#  Compile final list to email and dated ( so to track who i emailed) 

In [8]:
charity_filepath = 'D:\\Coding\\git_projects\\Charity_Scrape\\' + 'charity_email_sent_' + dti.now().strftime("%Y-%m-%d") + '.csv'
print(charity_filepath)
to_send.to_csv(charity_filepath)

D:\Coding\git_projects\Charity_Scrape\charity_email_sent_2024-02-08.csv


In [9]:
# c_data
to_send

,Charity name,Income,overview,what_it_does,email,website
0,the grange centre for people with disabilities,5002085,to maximise the independence of adults with ph...,people with disabilities,info@grangecentre.org.uk,www.grangecentre.org.uk
1,brambletye school trust limited,5008530,education/training of young people,children/young people,schooloffice@brambletye.com,www.brambletye.co.uk
2,treetops hospice trust,5016230,"treetops hospice provides daycare, complementa...",people with disabilities,info@treetopshospice.org.uk,www.treetopshospice.org.uk
3,the hammond school limited,5024030,the hammond school is one of 8 vocational seco...,children/young people,info@thehammondschool.co.uk,www.thehammondschool.co.uk
4,groundwork yorkshire limited,5055289,to bring about lasting environmental improveme...,children/young people,wakefield@groundwork.org.uk,https://www.groundwork.org.uk/hubs/north-east-...
...,...,...,...,...,...,...
95,kingsway clc trust,5769301,to advance the christian faith through the pro...,children/young people,sales@kingswayclctrust.com,http://www.kingswayclctrust.com
96,rikkyo school in england trust,5785251,the education in accordance with the christian...,children/young people,eikoku@rikkyo.uk,www.rikkyo.co.uk
97,british home and hospital for incurables,5788669,british home provides specialist nursing care ...,people with disabilities,enquiries@britishhome.org.uk,www.britishhome.org.uk
98,milton keynes community foundation limited,5798052,we are a local independant grant making charit...,general charitable purposes,info@mkcommunityfoundation.co.uk,www.mkcommunityfoundation.co.uk


if these are in original register, remove them

In [10]:

c_data.drop(labels=to_send.index, axis=0, inplace=True)

c_data.to_csv('D:/Coding/git_projects/Charity_Scrape/charity_email_final.csv')

c_data

,Charity name,Income,overview,what_it_does,email,website
100,ludgrove school trust limited,5834428,"the provision of education at ludgrove, a boar...",children/young people,office@ludgroveschool.co.uk,www.ludgrove.net
101,child poverty action group,5835230,"child poverty action group is a charity, worki...",children/young people,office@cpag.org.uk,www.cpag.org.uk
102,st nicholas school (harlow) limited,5841653,"operation of independent, charitable school",children/young people,accounts@saintnicholasschool.net,www.saintnicholasschool.net
103,richard shuttleworth trustees,5877000,sole trustee of richard ormonde shuttleworth r...,other defined groups,enquiries@shuttleworth.org,www.shuttleworth.org
104,northern consortium,5881027,northern consortium funds a range of education...,education/training,info@nccharity.org.uk,www.nccharity.org.uk
...,...,...,...,...,...,...
828,garfield weston foundation,84004000,the garfield weston foundation is a family-fou...,general charitable purposes,admin@garfieldweston.org,www.garfieldweston.org
829,the royal national institute of blind people,85709000,rnib is the uks leading charity offering infor...,children/young people,helpline@rnib.org.uk,www.rnib.org.uk
830,education development trust,89070000,the charity seeks to advance education for the...,children/young people,corpgov@educationdevelopmenttrust.com,http://www.educationdevelopmenttrust.com/
831,wateraid,91913000,"wateraid is an international not-for-profit, d...",the prevention or relief of poverty,supportercare@wateraid.org,www.wateraid.org
